In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
trainset = datasets.MNIST('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
valset = datasets.MNIST('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=60000, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

In [3]:
images, labels = next(iter(trainloader))
print(images.shape)
print(labels.shape)

torch.Size([60000, 1, 28, 28])
torch.Size([60000])


In [4]:
print(images.shape[0])
# print(images[0].flatten().shape[0])
# print(images[0].flatten().view(784,1))

60000


In [9]:
class AutoEncoders(nn.Module):
    sigmoid = nn.ReLU()
    size_input = images[0].flatten().shape[0]
    size_h = 128
    size_l = 32
    def __init__(self): 
        super().__init__()
        self.Wh1 = nn.Parameter(torch.randn(self.size_h,self.size_input,requires_grad = True,dtype = torch.float)/np.sqrt(self.size_input))
        self.bh1 = nn.Parameter(torch.randn(self.size_h,1,requires_grad = True,dtype = torch.float))
        self.Wf = nn.Parameter(torch.randn(self.size_l,self.size_h,requires_grad = True,dtype = torch.float)/np.sqrt(self.size_h))
        self.bf = nn.Parameter(torch.randn(self.size_l,1,requires_grad = True,dtype = torch.float))
        self.Wh2 = nn.Parameter(torch.randn(self.size_h,self.size_l,requires_grad = True,dtype = torch.float)/np.sqrt(self.size_l))
        self.bh2 = nn.Parameter(torch.randn(self.size_h,1,requires_grad = True,dtype = torch.float))
        self.Wg = nn.Parameter(torch.randn(self.size_input,self.size_h,requires_grad = True,dtype = torch.float)/np.sqrt(self.size_h))
        self.bg = nn.Parameter(torch.randn(self.size_input,1,requires_grad = True,dtype = torch.float))
        
    def forward(self,input):
        self.h1 = self.sigmoid(self.Wh1@input + self.bh1)
        self.f = self.sigmoid(self.Wf@self.h1 + self.bf)
        self.h2 = self.sigmoid(self.Wh2@self.f + self.bh2)
        self.g = self.sigmoid(self.Wg@self.h2 + self.bg)
        return self.g
        

In [10]:
def train_loop(model,loss_fn,optimizer,input):
    loss = 0
    for i in range(input.shape[0]):
        input_single = input[i].flatten().view(784,1)
        pred = model(input_single)
        loss_cal = loss_fn(pred,input_single)
        loss = loss + loss_cal
        optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss/input.shape[0]

In [11]:
model = AutoEncoders()
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)

nb_epochs = 20
loss_array = np.empty(shape = (1,1))
for i in range(nb_epochs):
    current_loss = train_loop(model,loss_fn,optimizer,images)
    print(current_loss)
#     loss_array = loss_array.cat((loss_array,loss),dim = 0)

tensor(2.5746, grad_fn=<DivBackward0>)
tensor(2.1104e+10, grad_fn=<DivBackward0>)
tensor(inf, grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<DivBackward0>)
tensor(nan, grad_fn=<DivBackward0>)


KeyboardInterrupt: 

In [ ]:
np.size(images[0].flatten().numpy())

In [ ]:
for i in range(64):
    plt.imshow(images[i].numpy().squeeze(), cmap='gray_r');
    print(labels[i])
    plt.show()